percent_true
has_ergot indicates if there is at least one true
sum_severity
previous_one_has_ergot indicates if there is at least one true last year
previous_one_severity
previous_two_has_ergot indicates if there is at least one true two years ago
previous_two_severity

we also need a car_uid column to link the prov - district in this table to the ag region boundary table

the final ergot table has

In [ ]:
import sqlalchemy as sq
import geopandas as gpd  # type: ignore
import pandas as pd  # type: ignore
from dotenv import load_dotenv
import os, sys

sys.path.append("../")
from Shared.DataService import DataService  # type: ignore

In [ ]:
load_dotenv()
PG_DB = os.getenv("POSTGRES_DB")
PG_ADDR = os.getenv("POSTGRES_ADDR")
PG_PORT = os.getenv("POSTGRES_PORT")
PG_USER = os.getenv("POSTGRES_USER")
PG_PW = os.getenv("POSTGRES_PW")

In [ ]:
db = DataService(PG_DB, PG_ADDR, PG_PORT, PG_USER, PG_PW)
conn = db.connect()

In [ ]:
query = "SELECT * FROM public.ergot_sample"
ergot = pd.read_sql_query(query, conn)  # type: ignore
#components = pd.DataFrame(query, columns=["soil_id", "poly_id"])  # type: ignore

In [ ]:
ergot

In [ ]:
# calculate the car_uid
ergot.loc[ergot['province'] == 'MB', 'car_uid'] = ergot.loc[ergot['province'] == 'MB', 'crop_district'] + 4600
ergot.loc[ergot['province'] == 'SK', 'car_uid'] = (ergot.loc[ergot['province'] == 'SK', 'crop_district'] - 1) + 4700 
ergot.loc[ergot['province'] == 'AB', 'car_uid'] = (ergot.loc[ergot['province'] == 'AB', 'crop_district'] * 10) + 4800

In [ ]:
ergot

In [ ]:
ergot[['car_uid']] = ergot[['car_uid']].astype(int)

In [ ]:
ergot

In [ ]:
ergot['percnt_true'] = None
ergot['has_ergot'] = False
ergot['sumSeverity'] = 0
ergot['present_prev1'] = False
ergot['present_prev2'] = False
ergot['present_prev3'] = False
ergot['present_in_neighbor'] = None

In [ ]:
for year in ergot['year'].unique():
    for uid in ergot['car_uid'].unique():
        relSamples = ergot.query(f'year == {year} and car_uid == {uid}')
        
        ergot.loc[(ergot['year'] == year) & (ergot['car_uid'] == uid), 'percnt_true'] = relSamples['incidence'].sum()/len(relSamples.index)
        ergot.loc[(ergot['year'] == year) & (ergot['car_uid'] == uid), 'has_ergot'] = relSamples['incidence'].sum() > 0
        ergot.loc[(ergot['year'] == year) & (ergot['car_uid'] == uid), 'sumSeverity'] = relSamples['severity'].sum()

        prev1Year = ergot.query(f'year == {year - 1} and car_uid == {uid}')
        prev2Year = ergot.query(f'year == {year - 2} and car_uid == {uid}')
        prev3Year = ergot.query(f'year == {year - 3} and car_uid == {uid}')
        ergot.loc[(ergot['year'] == year) & (ergot['car_uid'] == uid), 'present_prev1'] = prev1Year['incidence'].sum() > 0
        ergot.loc[(ergot['year'] == year) & (ergot['car_uid'] == uid), 'present_prev2'] = prev2Year['incidence'].sum() > 0
        ergot.loc[(ergot['year'] == year) & (ergot['car_uid'] == uid), 'present_prev3'] = prev3Year['incidence'].sum() > 0

In [ ]:
# bordering agRegion has Ergot

In [ ]:
ergot

In [ ]:
db.cleanup()